In [ ]:
import pandas as pd
#import tweepy
import json
import MeCab
from collections import Counter
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import re
import codecs
from datetime import timedelta

In [ ]:
access_token = 
access_token_secret = 
consumer_key = 
consumer_key_secret = 

#twitterAPIのためのconfigファイル
#JSON_LOAD_FILE = r"./json/twitter.json"
#twitterからデータを抽出するときの検索ワード
search_word = "野菜"
#twitterからデータを抽出するときの取得件数
SEARCH_COUNT = 100

#tweeterAPIと繋げるAPIインスタンスを生成する
def twitter_api_connect():
    #twiteerのAPIを利用するためのキーを取得


    #twiteerのAPIを利用するための、認証データ作成
    auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
    auth.set_access_token(access_token,access_token_secret)

    #twitterのAPIインスタンス生成
    twitter_api = tweepy.API(auth, wait_on_rate_limit = True)
    return twitter_api

#tweetのデータを取得し、リスト化し、返却する
def tweet_list_create(twitter_api, search_word, search_count):
    #カーソルを使用してデータ取得
    search_tweets = twitter_api.search_users(q = search_word, count = search_count,lang="ja")

    #tweetの内容を格納するためのリスト変数
    search_tweet_list = []

    #取得したtweetの内容をリストに格納
    for search_tweet in search_tweets:
        #print(search_tweets)
        search_tweet_list.append(search_tweet.description) 
    return search_tweet_list



#twitterのAPIインスタンス生成
api = twitter_api_connect()

#twitterからデータを取得
tweet_list = tweet_list_create(api, search_word , SEARCH_COUNT)
tweet_list

In [ ]:

#アカウント指定(@の後ろの英数字、↓はサンプルID)
Account = ''

#ツイート保存用変数&取得するツイートを計算する
tweet_data = []
num = 0

for page in range(17):
    tweets = api.user_timeline(screen_name=Account, count=200, page=page)
    for tweet in tweets:
        tweet.created_at += timedelta(hours=9)
#         print('----------') #見やすいように
#         print(tweet.created_at) #書くツイートの投稿時間表示
#         print(tweet.text) #各ツイート内容表示
        num += 1
        tweet_data.append(['@'+Account, tweet.created_at, tweet.text])

print(num, 'ツイート表示しました。')

# ツイートデータをデータフレームに変換
tweet_data = pd.DataFrame(tweet_data, columns=["account", "date", "tweet"])


In [ ]:
import pandas as pd

# 極性辞書をデータフレームで読み込み
df_dic = pd.read_csv('pn_ja.dic', sep=':', names=("Word", "読み", "品詞", "Score"), encoding='shift-jis')

keys = df_dic["Word"].tolist()
values = df_dic["Score"].tolist()
dic = dict(zip(keys, values))
dic

In [ ]:
import requests

url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
r = requests.get(url)
tmp = r.text.split('\r\n')
stopwords = []
for i in range(len(tmp)):
    if len(tmp[i]) < 1:
        continue
    stopwords.append(tmp[i])
print(stopwords)

#####joblibで保存して使いたい時     
import joblib
joblib.dump(stopwords, 'stopwords.jb', compress = 3)
#####

In [ ]:
import joblib
stopwords = joblib.load('stopwords.jb')
stopwords

In [ ]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *

t = Tokenizer()
tweet_list = []

for tweet in tweet_data['tweet']:
    # 複合名詞を抽出
    a = Analyzer(token_filters=[CompoundNounFilter()])
    #result = [token.base_form for token in a.analyze(tweet) if token.part_of_speech.split(',')[0] in ['名詞']]
    # 動詞と形容詞を抽出
    result = [token.base_form for token in t.tokenize(tweet)
       if token.part_of_speech.split(',')[0] in ['動詞']]
    
    result.extend(result2)
    result_list = [tweet, result]
    tweet_list.append(result_list)
tweets = tweet_list.copy()

In [ ]:
for tweet in tweet_list:
    for stopword in stopwords:
        tweet[1] = [i for i in tweet[1] if i != stopword]
    tweet_word_num = len(tweet[1])
    tweet.append(tweet_word_num)
    # 辞書で判定
    results = []
    for sentence in tweet[1]:
        word_score = []
        score = dic.get(sentence)
        word_score = (sentence, score)
        results.append(word_score)

    # 判定を追加
    tweet.append(results) 
    # 曲のスコアを2パターンで判定。「Noneを含むパターン」と「Noneを含まないパターン」
    score = 0
    score1 = 0
    score2 = 0
    for word in tweet[3]:
        if word[1] != None:
            score += word[1]
    cnt = sum(not (i[1] == None) for i in tweet[3])  # Noneを含まない形態素数
    if score != 0:
        score1 = score / tweet[2]  # スコアの合計 / 全形態素数
        score2 = score / cnt  # スコアの合計 / Noneを含まない形態素数
    # スコアを追加
    tweet.append(cnt)
    tweet.append(score1)
    tweet.append(score2)

In [ ]:
tubu = []
score_list = []
score_list2 = []

for i in range(len(tweet_list)):
    tubu.append(tweet_list[i][0])
    score_list.append(tweet_list[i][5])
    score_list2.append(tweet_list[i][6])

In [ ]:
#辞書に無い単語は無かったものとしてスコアを算出する（スコア1）
#辞書に無い単語はスコア 0 として算出する（スコア2）

np_df = pd.DataFrame(tubu)
np_df[1] = score_list
np_df[2] = score_list2
np_df.columns = ['text','score','score2']
np_df

In [ ]:
np_df = np_df[np_df['score'] != 0].reset_index(drop=True)
np_df

In [ ]:
np_df['score'].mean(),np_df['score2'].mean()

In [ ]:
word_list = []
for i in range(len(np_df)):
    word_list.extend(tweet_list[i][1])
word_list

In [ ]:
word_list = [re.sub('@.*', '', s) for s in word_list]
word_list = [re.sub('https.*', '', s) for s in word_list]

In [ ]:
a = Counter(word_list)
a.most_common()

In [ ]:
s_list = ['する','ん','てる','の','なる','いる','ある','言う','ない','いい','られる','しまう','','ー','w','(',')','`)','☺']

word_stoped_list =[w for w in word_list if w not in s_list]
word_stoped_list

In [ ]:

#wordcloud用のフォントパス
WORDCLOUD_FONT_PATH = 'C:\Windows\Fonts\yumin.ttf'

#Mac
#'/System/Library/Fonts/ヒラギノ明朝 ProN.ttc'

#Windows版
#'C:\Windows\Fonts\yumin.ttf'

#wordcloud用の幅
WORDCLOUD_WIDTH = 800
#wordcloud用の高さ
WORDCLOUD_HEIGHT = 500
#wordcloud用の背景色
WORDCLOUD_BG_COLOR = 'white'
#twitterからデータを抽出するときの検索ワード
SEARCH_WORD = "negapoji_動詞cloud"

def word_cloud_png_creater(word_list, font_path, width, height, bg_color, search_word):
    word_counter = Counter(word_list)
    wc = WordCloud(font_path=font_path,
                   width=width, 
                   height=height, 
                   background_color=bg_color).generate_from_frequencies(word_counter)
    wc.to_file('./photo/{}.png'.format(search_word))

#フォルダに、検索文字列.pngでwordcloud作成
word_cloud_png_creater(word_stoped_list, WORDCLOUD_FONT_PATH, WORDCLOUD_WIDTH, WORDCLOUD_HEIGHT, WORDCLOUD_BG_COLOR, SEARCH_WORD)

In [ ]:
import pandas as pd
file = pd.read_csv('./data/ネガポジUnknown.csv',encoding = 'ansi')

In [ ]:
plt.plot(file['score'])

参考
https://zatsugaku-engineer.com/python/negaposi/
https://qiita.com/ChabesuB/items/e451f9a3882433658b27